In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import xgboost as xgb
from datetime import datetime

# Load datasets
vessel_data = pd.read_csv('../ais_train.csv', sep='|')
schedule_data = pd.read_csv('../schedules_to_may_2024.csv', sep='|')
ports_data = pd.read_csv('../ports.csv', sep='|')  # Load ports data
vessels_info = pd.read_csv('../vessels.csv', sep='|')  # Load vessel specifications

# Display the first few rows of each dataset
display(vessel_data.head())
display(schedule_data.head())
display(ports_data.head())
display(vessels_info.head())

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId
0,2024-01-01 00:00:25,284.0,0.7,0,88,0,01-09 23:00,-34.74370,-57.85130,61e9f3a8b937134a3c4bfdf7,61d371c43aeaecc07011a37f
1,2024-01-01 00:00:36,109.6,0.0,-6,347,1,12-29 20:00,8.89440,-79.47939,61e9f3d4b937134a3c4bff1f,634c4de270937fc01c3a7689
2,2024-01-01 00:01:45,111.0,11.0,0,112,0,01-02 09:00,39.19065,-76.47567,61e9f436b937134a3c4c0131,61d3847bb7b7526e1adf3d19
3,2024-01-01 00:03:11,96.4,0.0,0,142,1,12-31 20:00,-34.41189,151.02067,61e9f3b4b937134a3c4bfe77,61d36f770a1807568ff9a126
4,2024-01-01 00:03:51,214.0,19.7,0,215,0,01-25 12:00,35.88379,-5.91636,61e9f41bb937134a3c4c0087,634c4de270937fc01c3a74f3


,vesselId,shippingLineId,shippingLineName,arrivalDate,sailingDate,portName,portId,portLatitude,portLongitude
0,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-10-02 00:00:00+00:00,2023-10-03 00:00:00+00:00,Port of Brunswick,61d38499b7b7526e1adf3d54,31.140556,-81.496667
1,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-10-27 00:00:00+00:00,2023-10-27 00:00:00+00:00,Port of Southampton,61d3832bb7b7526e1adf3b63,50.902500,-1.428889
2,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-10-19 00:00:00+00:00,2023-10-20 00:00:00+00:00,Port of Bremerhaven,61d375e793c6feb83e5eb3e2,53.563611,8.554722
3,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-10-09 00:00:00+00:00,2023-10-10 00:00:00+00:00,Port of New York,61d38481b7b7526e1adf3d23,40.688333,-74.028611
4,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-09-25 00:00:00+00:00,2023-09-26 00:00:00+00:00,Manzanillo International Terminal,61d37d0199db2ccf7339eee1,9.372370,-79.879790


,portId,name,portLocation,longitude,latitude,UN_LOCODE,countryName,ISO
0,61d36ed80a1807568ff9a064,Port of Algiers,Algiers,3.067222,36.773611,DZALG,Algeria,DZ
1,61d36ed80a1807568ff9a065,Port of Annaba,Annaba,7.772500,36.900556,DZAAE,Algeria,DZ
2,61d36edf0a1807568ff9a070,Port of Oran,Oran,-0.639722,35.712222,DZORN,Algeria,DZ
3,61d36ee00a1807568ff9a072,Port of Skikda,Skikda,6.905833,36.887500,DZSKI,Algeria,DZ
4,61d36ee10a1807568ff9a074,Port of Pago-Pago,Pago-Pago,-170.690556,-14.274167,ASPPG,American Samoa,AS


,shippingLineId,vesselId,CEU,DWT,GT,NT,vesselType,breadth,depth,draft,enginePower,freshWater,fuel,homePort,length,maxHeight,maxSpeed,maxWidth,rampCapacity,yearBuilt
0,61a8e672f9cba188601e84ab,61e9f38eb937134a3c4bfd8b,6500,21200.0,58684,17606.0,83.0,32.0,22.20,NaN,0.0,NaN,NaN,OSLO,199.00,5.0,18.6,15.2,150.0,2000
1,61ec94f1a8cafc0e93f0e92a,61e9f38eb937134a3c4bfd8d,4902,12325.0,46800,NaN,83.0,31.0,NaN,NaN,14220.0,NaN,NaN,MONROVIA,182.00,NaN,NaN,NaN,NaN,2006
2,61e213d5d612676a0f0fb755,61e9f38eb937134a3c4bfd8f,5000,13059.0,46800,NaN,83.0,31.0,NaN,NaN,14220.0,NaN,NaN,SAINT JOHN'S,182.00,NaN,NaN,NaN,NaN,2010
3,61be24574ea00ae59d0fe388,61e9f38eb937134a3c4bfd91,4200,12588.0,39362,NaN,83.0,28.0,NaN,NaN,11060.0,NaN,NaN,NaN,167.00,NaN,NaN,NaN,NaN,2011
4,61a8e673f9cba188601e84ae,61e9f390b937134a3c4bfd93,7450,21052.0,75528,24391.0,83.0,37.2,22.23,NaN,13140.0,491.47,3236.78,Panama,199.98,NaN,NaN,NaN,NaN,2018


In [9]:
print(schedule_data)

                        vesselId            shippingLineId  \
0       61e9f3b1b937134a3c4bfe53  61a8e672f9cba188601e84ac   
1       61e9f3b1b937134a3c4bfe53  61a8e672f9cba188601e84ac   
2       61e9f3b1b937134a3c4bfe53  61a8e672f9cba188601e84ac   
3       61e9f3b1b937134a3c4bfe53  61a8e672f9cba188601e84ac   
4       61e9f3b1b937134a3c4bfe53  61a8e672f9cba188601e84ac   
...                          ...                       ...   
136245  61e9f3abb937134a3c4bfe19  61a8e672f9cba188601e84ab   
136246  61e9f3abb937134a3c4bfe19  61a8e672f9cba188601e84ab   
136247  61e9f3abb937134a3c4bfe19  61a8e672f9cba188601e84ab   
136248  61e9f3a9b937134a3c4bfe01  61a8e672f9cba188601e84ab   
136249  61e9f3a9b937134a3c4bfe01  61a8e672f9cba188601e84ab   

                  shippingLineName                arrivalDate  \
0       Wallenius Wilhelmsen Ocean  2023-10-02 00:00:00+00:00   
1       Wallenius Wilhelmsen Ocean  2023-10-27 00:00:00+00:00   
2       Wallenius Wilhelmsen Ocean  2023-10-19 00:00:00+00:0

# Merging datasets

In [10]:
# Step 1: Convert 'time' column to datetime in vessel_data
vessel_data['time'] = pd.to_datetime(vessel_data['time'])

# Step 2: Get the latest vessel data for each vessel
latest_vessel_data = vessel_data.loc[vessel_data.groupby('vesselId')['time'].idxmax()]

# Select relevant columns from the latest vessel data
latest_vessel_data = latest_vessel_data[['vesselId', 'latitude', 'longitude', 'navstat', 'portId']]

# Step 3: Merge with schedule data on 'vesselId'
combined_data = pd.merge(schedule_data, latest_vessel_data, on='vesselId', how='left')

# Step 4: Display the resulting combined data
display(combined_data.head())

,vesselId,shippingLineId,shippingLineName,arrivalDate,sailingDate,portName,portId_x,portLatitude,portLongitude,latitude,longitude,navstat,portId_y
0,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-10-02 00:00:00+00:00,2023-10-03 00:00:00+00:00,Port of Brunswick,61d38499b7b7526e1adf3d54,31.140556,-81.496667,8.80907,-79.58758,0.0,61d37cff99db2ccf7339eedd
1,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-10-27 00:00:00+00:00,2023-10-27 00:00:00+00:00,Port of Southampton,61d3832bb7b7526e1adf3b63,50.902500,-1.428889,8.80907,-79.58758,0.0,61d37cff99db2ccf7339eedd
2,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-10-19 00:00:00+00:00,2023-10-20 00:00:00+00:00,Port of Bremerhaven,61d375e793c6feb83e5eb3e2,53.563611,8.554722,8.80907,-79.58758,0.0,61d37cff99db2ccf7339eedd
3,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-10-09 00:00:00+00:00,2023-10-10 00:00:00+00:00,Port of New York,61d38481b7b7526e1adf3d23,40.688333,-74.028611,8.80907,-79.58758,0.0,61d37cff99db2ccf7339eedd
4,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-09-25 00:00:00+00:00,2023-09-26 00:00:00+00:00,Manzanillo International Terminal,61d37d0199db2ccf7339eee1,9.372370,-79.879790,8.80907,-79.58758,0.0,61d37cff99db2ccf7339eedd


In [11]:
# Step 1: Merge the combined dataset with the ports data on 'portId'
combined_data = pd.merge(combined_data, ports_data[['portId', 'name', 'portLocation', 'longitude', 'latitude', 'countryName']], 
                          left_on='portId_x', right_on='portId', how='left')

# Step 2: Drop any redundant columns if necessary
# You can drop the original portId_x and keep the new portId from ports_data
combined_data.drop(columns=['portId_x','name',], inplace=True)

# Step 3: Display the updated combined dataset
display(combined_data)

,vesselId,shippingLineId,shippingLineName,arrivalDate,sailingDate,portName,portLatitude,portLongitude,latitude_x,longitude_x,navstat,portId_y,portId,portLocation,longitude_y,latitude_y,countryName
0,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-10-02 00:00:00+00:00,2023-10-03 00:00:00+00:00,Port of Brunswick,31.140556,-81.496667,8.80907,-79.58758,0.0,61d37cff99db2ccf7339eedd,61d38499b7b7526e1adf3d54,Brunswick,-81.496667,31.140556,United States
1,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-10-27 00:00:00+00:00,2023-10-27 00:00:00+00:00,Port of Southampton,50.902500,-1.428889,8.80907,-79.58758,0.0,61d37cff99db2ccf7339eedd,61d3832bb7b7526e1adf3b63,Southampton,-1.428889,50.902500,United Kingdom
2,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-10-19 00:00:00+00:00,2023-10-20 00:00:00+00:00,Port of Bremerhaven,53.563611,8.554722,8.80907,-79.58758,0.0,61d37cff99db2ccf7339eedd,61d375e793c6feb83e5eb3e2,Bremerhaven,8.554722,53.563611,Germany
3,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-10-09 00:00:00+00:00,2023-10-10 00:00:00+00:00,Port of New York,40.688333,-74.028611,8.80907,-79.58758,0.0,61d37cff99db2ccf7339eedd,61d38481b7b7526e1adf3d23,New York,-74.028611,40.688333,United States
4,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-09-25 00:00:00+00:00,2023-09-26 00:00:00+00:00,Manzanillo International Terminal,9.372370,-79.879790,8.80907,-79.58758,0.0,61d37cff99db2ccf7339eedd,61d37d0199db2ccf7339eee1,Colon,-79.879790,9.372370,Panama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136245,61e9f3abb937134a3c4bfe19,61a8e672f9cba188601e84ab,Hoegh Autoliners,2023-12-03 23:00:00+00:00,2023-12-03 23:00:00+00:00,Port of Laem Chabang,13.073889,100.895278,35.90857,141.00919,0.0,61d379911366c3998241d850,61d3802b29b60f6113c89f3d,Laem Chabang,100.895278,13.073889,Thailand
136246,61e9f3abb937134a3c4bfe19,61a8e672f9cba188601e84ab,Hoegh Autoliners,2023-11-14 23:00:00+00:00,2023-11-14 23:00:00+00:00,Port Zayed,24.525000,54.376111,35.90857,141.00919,0.0,61d379911366c3998241d850,61d38285b7b7526e1adf3a84,Abu Dhabi,54.376111,24.525000,United Arab Emirates
136247,61e9f3abb937134a3c4bfe19,61a8e672f9cba188601e84ab,Hoegh Autoliners,2023-11-15 23:00:00+00:00,2023-11-15 23:00:00+00:00,NaN,NaN,NaN,35.90857,141.00919,0.0,61d379911366c3998241d850,NaN,NaN,NaN,NaN,NaN
136248,61e9f3a9b937134a3c4bfe01,61a8e672f9cba188601e84ab,Hoegh Autoliners,2023-08-28 01:00:00+00:00,2023-08-30 17:00:00+00:00,The Port of Wilmington,39.716667,-75.521667,NaN,NaN,NaN,NaN,61d38528b7b7526e1adf3e6f,Wilmington,-75.521667,39.716667,United States


In [12]:
# Step 1: Merge the combined dataset with the vessels information on 'vesselId'
combined_data = pd.merge(combined_data, vessels_info[['vesselId', 'shippingLineId', 'CEU', 'DWT', 
                                                       'GT', 'vesselType', 'length', 'maxSpeed']], 
                          on='vesselId', how='left')

# Step 2: Drop any redundant columns if necessary
# For instance, if there are multiple columns for shippingLineId, you may want to drop duplicates.
# combined_data.drop(columns=['redundant_column'], inplace=True)  # Adjust based on actual redundancy

# Step 3: Display the updated combined dataset
display(combined_data.head())

,vesselId,shippingLineId_x,shippingLineName,arrivalDate,sailingDate,portName,portLatitude,portLongitude,latitude_x,longitude_x,...,longitude_y,latitude_y,countryName,shippingLineId_y,CEU,DWT,GT,vesselType,length,maxSpeed
0,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-10-02 00:00:00+00:00,2023-10-03 00:00:00+00:00,Port of Brunswick,31.140556,-81.496667,8.80907,-79.58758,...,-81.496667,31.140556,United States,61a8e672f9cba188601e84ac,7934.0,31143.0,74258.0,83.0,231.0,NaN
1,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-10-27 00:00:00+00:00,2023-10-27 00:00:00+00:00,Port of Southampton,50.902500,-1.428889,8.80907,-79.58758,...,-1.428889,50.902500,United Kingdom,61a8e672f9cba188601e84ac,7934.0,31143.0,74258.0,83.0,231.0,NaN
2,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-10-19 00:00:00+00:00,2023-10-20 00:00:00+00:00,Port of Bremerhaven,53.563611,8.554722,8.80907,-79.58758,...,8.554722,53.563611,Germany,61a8e672f9cba188601e84ac,7934.0,31143.0,74258.0,83.0,231.0,NaN
3,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-10-09 00:00:00+00:00,2023-10-10 00:00:00+00:00,Port of New York,40.688333,-74.028611,8.80907,-79.58758,...,-74.028611,40.688333,United States,61a8e672f9cba188601e84ac,7934.0,31143.0,74258.0,83.0,231.0,NaN
4,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-09-25 00:00:00+00:00,2023-09-26 00:00:00+00:00,Manzanillo International Terminal,9.372370,-79.879790,8.80907,-79.58758,...,-79.879790,9.372370,Panama,61a8e672f9cba188601e84ac,7934.0,31143.0,74258.0,83.0,231.0,NaN


In [13]:
print(combined_data)

                        vesselId          shippingLineId_x  \
0       61e9f3b1b937134a3c4bfe53  61a8e672f9cba188601e84ac   
1       61e9f3b1b937134a3c4bfe53  61a8e672f9cba188601e84ac   
2       61e9f3b1b937134a3c4bfe53  61a8e672f9cba188601e84ac   
3       61e9f3b1b937134a3c4bfe53  61a8e672f9cba188601e84ac   
4       61e9f3b1b937134a3c4bfe53  61a8e672f9cba188601e84ac   
...                          ...                       ...   
136245  61e9f3abb937134a3c4bfe19  61a8e672f9cba188601e84ab   
136246  61e9f3abb937134a3c4bfe19  61a8e672f9cba188601e84ab   
136247  61e9f3abb937134a3c4bfe19  61a8e672f9cba188601e84ab   
136248  61e9f3a9b937134a3c4bfe01  61a8e672f9cba188601e84ab   
136249  61e9f3a9b937134a3c4bfe01  61a8e672f9cba188601e84ab   

                  shippingLineName                arrivalDate  \
0       Wallenius Wilhelmsen Ocean  2023-10-02 00:00:00+00:00   
1       Wallenius Wilhelmsen Ocean  2023-10-27 00:00:00+00:00   
2       Wallenius Wilhelmsen Ocean  2023-10-19 00:00:00+00:0

In [14]:
# Check the data types of each column
print(combined_data.dtypes)

vesselId             object
shippingLineId_x     object
shippingLineName     object
arrivalDate          object
sailingDate          object
portName             object
portLatitude        float64
portLongitude       float64
latitude_x          float64
longitude_x         float64
navstat             float64
portId_y             object
portId               object
portLocation         object
longitude_y         float64
latitude_y          float64
countryName          object
shippingLineId_y     object
CEU                 float64
DWT                 float64
GT                  float64
vesselType          float64
length              float64
maxSpeed            float64
dtype: object


# Feature Engineering

In [15]:
# Convert arrivalDate column to datetime
combined_data['arrivalDate'] = pd.to_datetime(combined_data['arrivalDate'])

# Create essential time-based features from the arrivalDate
combined_data['hour'] = combined_data['arrivalDate'].dt.hour
combined_data['day_of_week'] = combined_data['arrivalDate'].dt.dayofweek
combined_data['month'] = combined_data['arrivalDate'].dt.month

# Optional: You can also create features from sailingDate if relevant
combined_data['sailingDate'] = pd.to_datetime(combined_data['sailingDate'])
combined_data['sailing_hour'] = combined_data['sailingDate'].dt.hour
combined_data['sailing_day_of_week'] = combined_data['sailingDate'].dt.dayofweek
combined_data['sailing_month'] = combined_data['sailingDate'].dt.month

# Handle missing values

In [17]:
# Fill missing values only for numeric columns
numeric_cols = combined_data.select_dtypes(include=[np.number]).columns
combined_data[numeric_cols] = combined_data[numeric_cols].fillna(combined_data[numeric_cols].mean())

# Optionally, handle categorical columns separately
# For example, for categorical columns, you could fill with the mode
categorical_cols = combined_data.select_dtypes(include=['object']).columns
for col in categorical_cols:
    combined_data[col].fillna(combined_data[col].mode()[0], inplace=True)

In [19]:
# Print all column names to check the actual structure
print(combined_data.columns.tolist())

['vesselId', 'shippingLineId_x', 'shippingLineName', 'arrivalDate', 'sailingDate', 'portName', 'portLatitude', 'portLongitude', 'latitude_x', 'longitude_x', 'navstat', 'portId_y', 'portId', 'portLocation', 'longitude_y', 'latitude_y', 'countryName', 'shippingLineId_y', 'CEU', 'DWT', 'GT', 'vesselType', 'length', 'maxSpeed', 'hour', 'day_of_week', 'month', 'sailing_hour', 'sailing_day_of_week', 'sailing_month']


# Training DWT Accidentally

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Select a small sample from the combined data
small_data = combined_data.sample(n=100, random_state=42)  # Randomly select 100 rows

# Define features (X) and target variable (y)
# Assuming you want to predict 'DWT_y' as an example target variable
X = small_data[['hour', 'day_of_week', 'month', 'CEU', 'length', 'maxSpeed']]
y = small_data['DWT']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a simple model (e.g., Linear Regression)
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')


Mean Squared Error: 3524623.3243412264
R-squared: 0.9252802610275043


In [21]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Assume combined_data is already prepared as before

# Define features (X) and target variable (y)
X = small_data[['hour', 'day_of_week', 'month', 'CEU', 'length', 'maxSpeed']]
y = small_data['DWT']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the XGBoost model
xgboost_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)

# Train the model
xgboost_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = xgboost_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')


Mean Squared Error: 621176.0678031876
R-squared: 0.9868314683950261


# Training Vessel Position Over Time

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Assume combined_data is already prepared and contains necessary columns

# Select a small sample from the combined data
small_data = combined_data.sample(n=100, random_state=42)  # Randomly select 100 rows

# Create features (X) for predicting latitude and longitude
# You can include previous positions, current time, and other relevant features
X = small_data[['hour', 'day_of_week', 'month', 'CEU', 'length', 'maxSpeed']]

# Define target variables (latitude and longitude)
y_latitude = small_data['latitude_x']  # Assuming latitude_x is the current position
y_longitude = small_data['longitude_x']  # Assuming longitude_x is the current position

# Split the data into training and testing sets for latitude
X_train_lat, X_test_lat, y_train_lat, y_test_lat = train_test_split(X, y_latitude, test_size=0.2, random_state=42)

# Train a model for latitude prediction
model_latitude = LinearRegression()
model_latitude.fit(X_train_lat, y_train_lat)

# Make predictions on the test set for latitude
y_pred_latitude = model_latitude.predict(X_test_lat)

# Evaluate latitude model
mse_latitude = mean_squared_error(y_test_lat, y_pred_latitude)
r2_latitude = r2_score(y_test_lat, y_pred_latitude)

print(f'Mean Squared Error (Latitude): {mse_latitude}')
print(f'R-squared (Latitude): {r2_latitude}')

# Repeat the same for longitude
X_train_lon, X_test_lon, y_train_lon, y_test_lon = train_test_split(X, y_longitude, test_size=0.2, random_state=42)

# Train a model for longitude prediction
model_longitude = LinearRegression()
model_longitude.fit(X_train_lon, y_train_lon)

# Make predictions on the test set for longitude
y_pred_longitude = model_longitude.predict(X_test_lon)

# Evaluate longitude model
mse_longitude = mean_squared_error(y_test_lon, y_pred_longitude)
r2_longitude = r2_score(y_test_lon, y_pred_longitude)

print(f'Mean Squared Error (Longitude): {mse_longitude}')
print(f'R-squared (Longitude): {r2_longitude}')


Mean Squared Error (Latitude): 832.9538303793464
R-squared (Latitude): -0.007508975682420926
Mean Squared Error (Longitude): 4258.515782107763
R-squared (Longitude): -0.007712919203008539


In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score

# Assume combined_data is already prepared and contains necessary columns

# Select a small sample from the combined data
small_data = combined_data.sample(n=1000, random_state=42)  # Randomly select 100 rows

# Create features (X) for predicting latitude and longitude
X = small_data[['hour', 'day_of_week', 'month', 'CEU', 'length', 'maxSpeed']]

# Define target variables (latitude and longitude)
y_latitude = small_data['latitude_x']  # Current latitude position
y_longitude = small_data['longitude_x']  # Current longitude position

# Split the data into training and testing sets for latitude
X_train_lat, X_test_lat, y_train_lat, y_test_lat = train_test_split(X, y_latitude, test_size=0.2, random_state=42)

# Train an XGBoost model for latitude prediction
model_latitude = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)
model_latitude.fit(X_train_lat, y_train_lat)

# Make predictions on the test set for latitude
y_pred_latitude = model_latitude.predict(X_test_lat)

# Evaluate latitude model
mse_latitude = mean_squared_error(y_test_lat, y_pred_latitude)
r2_latitude = r2_score(y_test_lat, y_pred_latitude)

print(f'Mean Squared Error (Latitude): {mse_latitude}')
print(f'R-squared (Latitude): {r2_latitude}')

# Repeat the same for longitude
X_train_lon, X_test_lon, y_train_lon, y_test_lon = train_test_split(X, y_longitude, test_size=0.2, random_state=42)

# Train an XGBoost model for longitude prediction
model_longitude = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)
model_longitude.fit(X_train_lon, y_train_lon)

# Make predictions on the test set for longitude
y_pred_longitude = model_longitude.predict(X_test_lon)

# Evaluate longitude model
mse_longitude = mean_squared_error(y_test_lon, y_pred_longitude)
r2_longitude = r2_score(y_test_lon, y_pred_longitude)

print(f'Mean Squared Error (Longitude): {mse_longitude}')
print(f'R-squared (Longitude): {r2_longitude}')


Mean Squared Error (Latitude): 581.4572818699636
R-squared (Latitude): 0.21720521280868688
Mean Squared Error (Longitude): 4975.154868562738
R-squared (Longitude): 0.24897236483223562


In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score

# Create features (X) for predicting latitude and longitude
X = combined_data[['hour', 'day_of_week', 'month', 'CEU', 'length', 'maxSpeed']]

# Define target variables (latitude and longitude)
y_latitude = combined_data['latitude_x']  # Current latitude position
y_longitude = combined_data['longitude_x']  # Current longitude position

# Split the data into training and testing sets for latitude
X_train_lat, X_test_lat, y_train_lat, y_test_lat = train_test_split(X, y_latitude, test_size=0.2, random_state=42)

# Train an XGBoost model for latitude prediction
model_latitude = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)
model_latitude.fit(X_train_lat, y_train_lat)

# Make predictions on the test set for latitude
y_pred_latitude = model_latitude.predict(X_test_lat)

# Evaluate latitude model
mse_latitude = mean_squared_error(y_test_lat, y_pred_latitude)
r2_latitude = r2_score(y_test_lat, y_pred_latitude)

print(f'Mean Squared Error (Latitude): {mse_latitude}')
print(f'R-squared (Latitude): {r2_latitude}')

# Repeat the same for longitude
X_train_lon, X_test_lon, y_train_lon, y_test_lon = train_test_split(X, y_longitude, test_size=0.2, random_state=42)

# Train an XGBoost model for longitude prediction
model_longitude = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)
model_longitude.fit(X_train_lon, y_train_lon)

# Make predictions on the test set for longitude
y_pred_longitude = model_longitude.predict(X_test_lon)

# Evaluate longitude model
mse_longitude = mean_squared_error(y_test_lon, y_pred_longitude)
r2_longitude = r2_score(y_test_lon, y_pred_longitude)

print(f'Mean Squared Error (Longitude): {mse_longitude}')
print(f'R-squared (Longitude): {r2_longitude}')


Mean Squared Error (Latitude): 342.03242545444664
R-squared (Latitude): 0.5433480261682493
Mean Squared Error (Longitude): 2978.7644372593672
R-squared (Longitude): 0.501212699067977
